In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline
import matplotlib
#matplotlib.use('agg')
matplotlib.style.use('ggplot')
import pickle as pkl
from matplotlib import pyplot as plt
from collections import Counter
from functools import reduce
import random
random.seed(1991)
import glob

In [2]:
dfm_dec=pd.read_csv("/data/dharp/compounding/datasets/DFM_Contextual_Temporal.csv",sep='\t')
dfm_dec

,modifier,head,1800_local_mi,1810_local_mi,1820_local_mi,1830_local_mi,1840_local_mi,1850_local_mi,1860_local_mi,1870_local_mi,...,1900_sim_with_modifier,1910_sim_with_modifier,1920_sim_with_modifier,1930_sim_with_modifier,1940_sim_with_modifier,1950_sim_with_modifier,1960_sim_with_modifier,1970_sim_with_modifier,1980_sim_with_modifier,1990_sim_with_modifier
0,a_n,aaaaa_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.073,0.055,0.062,0.044
1,a_n,addison_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.015,0.032,0.024
2,a_n,anti_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.216,0.139,0.132,0.109
3,a_n,archive_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.059,0.087,0.052,0.027
4,a_n,atom_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.121,0.122,0.202,0.123,0.101,0.121
5,a_n,b_n,0.007,0.008,0.006,0.006,0.007,0.008,0.011,0.012,...,0.982,0.946,0.978,0.924,0.941,0.669,0.815,0.479,0.258,0.319
6,a_n,band_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.055,0.035,0.040,0.043
7,a_n,baseball_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.018
8,a_n,behavior_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.055,0.117,0.030
9,a_n,bell_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.018,0.020,0.069,0.079


In [3]:
novel_compounds_list = pkl.load( open( "/data/dharp/compounding/datasets/novel_compounds_list.pkl", "rb" ) )
modifiers, heads = zip(*novel_compounds_list)
heads_list=list(set(heads))
modifiers_list=list(set(modifiers))

In [4]:
novel_compounds=pd.DataFrame(novel_compounds_list)
novel_compounds.columns=['modifier','head']
novel_compounds

,modifier,head
0,a_n,peaceful_n
1,a_n,robinson_n
2,a_n,supply_n
3,india_n,peaceful_n
4,india_n,standard_n
5,india_n,award_n
6,india_n,joint_n
7,india_n,companion_n
8,india_n,foundation_n
9,india_n,gentleman_n


In [5]:
modifiers=dfm_dec.loc[dfm_dec.modifier.isin(modifiers_list)].drop(['head','1800_sim_with_head','1810_sim_with_head',
'1820_sim_with_head','1830_sim_with_head','1840_sim_with_head','1850_sim_with_head','1860_sim_with_head','1870_sim_with_head',
 '1880_sim_with_head','1890_sim_with_head','1900_sim_with_head','1910_sim_with_head','1920_sim_with_head','1930_sim_with_head','1940_sim_with_head',
 '1950_sim_with_head','1960_sim_with_head','1970_sim_with_head','1980_sim_with_head','1990_sim_with_head'],axis=1)
modifier_features=modifiers.groupby(['modifier']).agg({'mean','std'})
modifier_features_1=modifier_features.columns.get_level_values(0)
modifier_features_2=modifier_features.columns.get_level_values(1)

cur_year=0
new_columns=[]
for year in modifier_features_1:
    new_columns.append("modifier_"+str(year)+"_"+modifier_features_2[cur_year])
    cur_year+=1
modifier_features.columns=new_columns
modifier_features.fillna(modifier_features.mean(),inplace=True)
modifier_features.reset_index(inplace=True)
modifier_features

,modifier,modifier_1800_local_mi_mean,modifier_1800_local_mi_std,modifier_1810_local_mi_mean,modifier_1810_local_mi_std,modifier_1820_local_mi_mean,modifier_1820_local_mi_std,modifier_1830_local_mi_mean,modifier_1830_local_mi_std,modifier_1840_local_mi_mean,...,modifier_1950_sim_with_modifier_mean,modifier_1950_sim_with_modifier_std,modifier_1960_sim_with_modifier_mean,modifier_1960_sim_with_modifier_std,modifier_1970_sim_with_modifier_mean,modifier_1970_sim_with_modifier_std,modifier_1980_sim_with_modifier_mean,modifier_1980_sim_with_modifier_std,modifier_1990_sim_with_modifier_mean,modifier_1990_sim_with_modifier_std
0,a_n,0.006,0.001,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.021,0.067,0.036,0.073,0.035,0.069,0.039,0.066,0.045,0.074
1,aaa_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.140,0.348,0.145,0.342,0.147,0.341,0.159,0.330,0.171,0.298
2,aarhus_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.040,0.022
3,abbreviated_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.167,0.408,0.167,0.408,0.209,0.350,0.383,0.251,0.394,0.140
4,abduction_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.000,0.000,0.000,0.000,0.000,0.000,0.166,0.371,0.374,0.255
5,abecedarian_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.000,0.178,0.000,0.206,0.000,0.225,0.000,0.238,0.983,0.237
6,abilities_n,0.006,0.002,0.005,0.001,0.004,0.001,0.003,0.000,0.003,...,0.108,0.229,0.176,0.213,0.152,0.216,0.145,0.259,0.132,0.271
7,ability_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.075,0.170,0.077,0.186,0.079,0.168,0.117,0.207,0.159,0.236
8,abnormalities_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.000,0.178,0.000,0.206,0.000,0.225,0.759,0.238,0.765,0.237
9,abolition_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.063,0.139,0.112,0.172,0.100,0.173,0.084,0.157,0.101,0.153


In [6]:
heads=dfm_dec.loc[dfm_dec['head'].isin(heads_list)].drop(['modifier','1800_sim_with_modifier', '1810_sim_with_modifier',
 '1820_sim_with_modifier','1830_sim_with_modifier','1840_sim_with_modifier','1850_sim_with_modifier','1860_sim_with_modifier',
 '1870_sim_with_modifier','1880_sim_with_modifier','1890_sim_with_modifier','1900_sim_with_modifier','1910_sim_with_modifier',
 '1920_sim_with_modifier','1930_sim_with_modifier','1940_sim_with_modifier','1950_sim_with_modifier','1960_sim_with_modifier',
 '1970_sim_with_modifier','1980_sim_with_modifier','1990_sim_with_modifier'],axis=1)
head_features=heads.groupby(['head']).agg({'mean','std'})
head_features_1=head_features.columns.get_level_values(0)
head_features_2=head_features.columns.get_level_values(1)

cur_year=0
new_columns=[]
for year in head_features_1:
    new_columns.append("head_"+str(year)+"_"+head_features_2[cur_year])
    cur_year+=1
head_features.columns=new_columns
head_features.fillna(head_features.mean(),inplace=True)
head_features.reset_index(inplace=True)
head_features

,head,head_1800_local_mi_mean,head_1800_local_mi_std,head_1810_local_mi_mean,head_1810_local_mi_std,head_1820_local_mi_mean,head_1820_local_mi_std,head_1830_local_mi_mean,head_1830_local_mi_std,head_1840_local_mi_mean,...,head_1950_sim_with_head_mean,head_1950_sim_with_head_std,head_1960_sim_with_head_mean,head_1960_sim_with_head_std,head_1970_sim_with_head_mean,head_1970_sim_with_head_std,head_1980_sim_with_head_mean,head_1980_sim_with_head_std,head_1990_sim_with_head_mean,head_1990_sim_with_head_std
0,a_n,0.006,0.001,0.005,0.001,0.004,0.000,0.003,0.000,0.003,...,0.117,0.244,0.168,0.285,0.190,0.295,0.212,0.308,0.222,0.307
1,aaron_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.249,0.233,0.236,0.262,0.192,0.215,0.219,0.264,0.187,0.161
2,ab_n,0.006,0.001,0.006,0.004,0.004,0.003,0.004,0.002,0.003,...,0.205,0.257,0.232,0.250,0.189,0.244,0.157,0.235,0.188,0.263
3,abandonment_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.040,0.119,0.092,0.275,0.144,0.303,0.213,0.147,0.214,0.174
4,abbreviation_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.038,0.193,0.110,0.263,0.127,0.226,0.211,0.229,0.216,0.219
5,abc_n,0.006,0.000,0.005,0.001,0.004,0.000,0.003,0.000,0.003,...,0.132,0.254,0.152,0.268,0.075,0.149,0.030,0.061,0.100,0.176
6,abduction_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.000,0.000,0.178,0.311,0.132,0.349,0.220,0.347,0.223,0.335
7,abdul_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.174,0.294,0.146,0.231,0.133,0.190,0.179,0.244,0.143,0.190
8,abdullah_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.268,0.403,0.317,0.375,0.312,0.357,0.296,0.351,0.301,0.364
9,ability_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,0.003,...,0.044,0.142,0.061,0.171,0.068,0.178,0.079,0.156,0.091,0.123


## Positive Set

In [7]:
positive_df=pd.merge(novel_compounds,head_features,on=["head"])
positive_df=pd.merge(positive_df,modifier_features,on=["modifier"])
positive_df['Plausibility']=True
positive_df

,modifier,head,head_1800_local_mi_mean,head_1800_local_mi_std,head_1810_local_mi_mean,head_1810_local_mi_std,head_1820_local_mi_mean,head_1820_local_mi_std,head_1830_local_mi_mean,head_1830_local_mi_std,...,modifier_1950_sim_with_modifier_std,modifier_1960_sim_with_modifier_mean,modifier_1960_sim_with_modifier_std,modifier_1970_sim_with_modifier_mean,modifier_1970_sim_with_modifier_std,modifier_1980_sim_with_modifier_mean,modifier_1980_sim_with_modifier_std,modifier_1990_sim_with_modifier_mean,modifier_1990_sim_with_modifier_std,Plausibility
0,a_n,peaceful_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.067,0.036,0.073,0.035,0.069,0.039,0.066,0.045,0.074,True
1,a_n,robinson_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.067,0.036,0.073,0.035,0.069,0.039,0.066,0.045,0.074,True
2,a_n,supply_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.067,0.036,0.073,0.035,0.069,0.039,0.066,0.045,0.074,True
3,india_n,peaceful_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.175,0.142,0.175,0.148,0.176,0.136,0.165,0.127,0.158,True
4,india_n,standard_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.175,0.142,0.175,0.148,0.176,0.136,0.165,0.127,0.158,True
5,india_n,award_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.175,0.142,0.175,0.148,0.176,0.136,0.165,0.127,0.158,True
6,india_n,joint_n,0.006,0.001,0.005,0.001,0.004,0.001,0.004,0.001,...,0.175,0.142,0.175,0.148,0.176,0.136,0.165,0.127,0.158,True
7,india_n,companion_n,0.006,0.001,0.005,0.001,0.004,0.001,0.004,0.001,...,0.175,0.142,0.175,0.148,0.176,0.136,0.165,0.127,0.158,True
8,india_n,foundation_n,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.175,0.142,0.175,0.148,0.176,0.136,0.165,0.127,0.158,True
9,india_n,gentleman_n,0.007,0.010,0.006,0.008,0.005,0.007,0.004,0.006,...,0.175,0.142,0.175,0.148,0.176,0.136,0.165,0.127,0.158,True


## Negative Sets

In [8]:
def neg_df_creator(file):
    pkl_file=pkl.load( open(file,'rb'))
    df=pd.DataFrame(pkl_file)
    df.columns=['modifier','head']
    negative_df=pd.merge(df,head_features,on=["head"])
    negative_df=pd.merge(negative_df,modifier_features,on=["modifier"])
    negative_df['Plausibility']=False
    return negative_df

In [9]:
def df_joiner(files):
    df_list=[]
    for file in files:
        neg_df=neg_df_creator(file)
        whole_df=pd.concat([neg_df,positive_df])
        df_list.append(whole_df)
    
    return df_list

In [10]:
corrupt_modifier_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_modifier*"):
    corrupt_modifier_files.append(file)
corrupt_modifiers=df_joiner(corrupt_modifier_files)

In [11]:
corrupt_head_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_head*"):
    corrupt_head_files.append(file)
corrupt_heads=df_joiner(corrupt_head_files)

## XGBoost

In [12]:
acc_ch=[]
for i,corrupt_head in enumerate(corrupt_heads):
    data=corrupt_head.drop(['modifier','head'],axis=1)
    X, Y = data.iloc[:,:-1],data.iloc[:,-1]
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1991)
    # fit model no training data
    model = XGBClassifier(njobs=-1)
    model.fit(X_train, y_train)
    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    print("DF",i+1)
    accuracy = accuracy_score(y_test, predictions)
    acc_ch.append(accuracy)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))


DF 1
Accuracy: 71.66%
DF 2
Accuracy: 71.50%
DF 3
Accuracy: 71.90%
DF 4
Accuracy: 71.42%
DF 5
Accuracy: 71.78%
DF 6
Accuracy: 72.14%
DF 7
Accuracy: 71.45%
DF 8
Accuracy: 71.46%
DF 9
Accuracy: 71.42%
DF 10
Accuracy: 70.93%


In [13]:
round(np.mean(acc_ch)*100,2)

71.57

In [14]:
round(np.std(acc_ch)*100,2)

0.31

In [15]:
acc_cm=[]
for i,corrupt_head in enumerate(corrupt_modifiers):
    data=corrupt_head.drop(['modifier','head'],axis=1)
    X, Y = data.iloc[:,:-1],data.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1991)
# fit model no training data
    model = XGBClassifier(njobs=-1)
    model.fit(X_train, y_train)
# make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
# evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    acc_cm.append(accuracy)
    print("DF",i+1)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))

DF 1
Accuracy: 68.97%
DF 2
Accuracy: 68.80%
DF 3
Accuracy: 68.74%
DF 4
Accuracy: 68.36%
DF 5
Accuracy: 69.08%
DF 6
Accuracy: 68.79%
DF 7
Accuracy: 68.68%
DF 8
Accuracy: 68.51%
DF 9
Accuracy: 68.86%
DF 10
Accuracy: 68.84%


In [16]:
round(np.mean(acc_cm)*100,2)

68.76

In [17]:
round(np.std(acc_cm)*100,2)

0.2